<a href="https://colab.research.google.com/github/raulbenitez/tortuosity/blob/master/Final%20version%20of%20the%20Automatic%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import pydicom
import os
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import glob
import mahotas as mh
import scipy.ndimage
import skimage as sk
import cv2
from skimage import morphology
from skimage import measure
from skimage import img_as_ubyte,img_as_int
from skimage import segmentation, graph
from skimage import img_as_float
from skimage.filters import threshold_otsu
from skimage.filters import median
from skimage.measure import label, regionprops
from skimage.transform import resize
from skimage.segmentation import clear_border
from skimage.util import invert
from skimage.morphology import skeletonize, disk,binary_dilation,binary_closing
from skimage.morphology import erosion, dilation
from skimage.morphology import disk
from skan import draw
from scipy.ndimage import gaussian_filter
from scipy import ndimage as nd
from sklearn.cluster import KMeans
from PIL import Image as img
from skan import draw
from skan import skeleton_to_csgraph, summarize, Skeleton
from matplotlib import gridspec

Load data from google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Access to all the DICOM files of the folder

In [ ]:
root_folder = '//content/drive/My Drive/3_Reconstruction'
l=[]
for root, dirs, files in os.walk(root_folder, topdown=False):
  print(files)
  for name in files:
      file_path = os.path.join(root,name)
      #print(file_path)
      if (name.endswith('3.1.dcm')):
          l.append(file_path)

In [ ]:
def pre_processing(path):
    f=[] 
    ds = pydicom.dcmread(path) # Read the DICOM file
    image2D=ds.pixel_array
    f.append(image2D.astype(np.int16)) # Append all of its pixels to a 2D array
    print('DICOM DESCRIPTION:')        # Extract patient information from Metadata
    print(' -Patient Sex: %s' %ds.PatientSex)
    print(' -Patient ID: %s' %ds.PatientID)
    print(' -Pixel Spacing: %s' %ds.PixelSpacing)
    print('\n')
    pixspac=ds.PixelSpacing[0]  # Save the pixel spacing attribute in a variable for easy use 
   
    
    # Saving the original image
    folder_dir=os.path.dirname(path)
    original_image_ubyte=img_as_ubyte(f[0])
    cv2.imwrite(os.path.join(folder_dir,'Original image.jpg'),original_image_ubyte)
    
    I_train=f[0] # Make copy of the original image 
    I_train_float = img_as_float(I_train) 
    # Reshape training image as 1d array, for the Kmeans segmentantion:
    v_I_train = I_train_float.reshape(I_train_float.shape[0]*I_train_float.shape[1],1).astype(np.float) 

    # Fit a Kmeans clustering model with two clusters using training image:
    kmeans= KMeans(n_clusters=3, random_state=0).fit(v_I_train)

    v_kmeans = kmeans.predict(v_I_train)
    I_kmeans = np.array(v_kmeans).reshape(f[0].shape[0],f[0].shape[1]).astype(np.uint8) # Shape back result as image 
    
    I_kmeans_16=img_as_int(I_kmeans)
    cv2.imwrite(os.path.join(folder_dir,'Kmeans.jpg'),I_kmeans_16)
    # Selection of the cluster corresponding to the aorta 
    index=np.argmax(kmeans.cluster_centers_)
    aorta=I_kmeans==index
    return aorta,f[0]

def segmentation(aorta,path):  
    # Start morphological operation process 
    folder_dir=os.path.dirname(path)
    ds = pydicom.dcmread(path)
    pixspac=ds.PixelSpacing[0] # Obtain again the attribute pixel spacing from the path
    
    selem = disk(3)  # Create the structuring element of disk of radius 3
    eroded = erosion(aorta, selem)       # apply erosion
    dilated = dilation(eroded, selem)    # apply dilation
    dilated = dilation(dilated, selem)   # apply dilation
    eroded = erosion(dilated, selem)     # apply erosion
    cv2.imwrite(os.path.join(folder_dir,'morphological_operations.jpg'),eroded*255)
        
    # Labeling process
    label_image, nregions = label(eroded,return_num=True)

    # Deleting non-interest regions by its area
    props=measure.regionprops(label_image)  # Extract the properties of each area
    va = []
    for prop in props:
        va.append(prop.area)
    indkeep = np.array(np.argmax(va))
    v = np.arange(nregions)
    vkill = np.setdiff1d(v,indkeep)+1 
    I2 = eroded
    for kkill in vkill:
        I2[label_image==kkill]=0     # Set to 0 (background) the non interest areas
    
    # Making the contour more regular
    selem = disk(1)       # Structuring element disk of radius 1
    dilated = dilation(I2, selem)      # Dilation of radius 1
    closed=binary_closing(dilated,selem)    # Binary closing
    cv2.imwrite(os.path.join(folder_dir,'final_segmented_structure.jpg'),closed*255)

    closed_int=img_as_ubyte(closed)
    props1=measure.regionprops(closed_int)   # Extract properties from the remaining structure to pass the 1st filter

    # FIRST FILTER
    # Removing bad reconstruction cases by its major length, the orientation and the position of the centroid

    if (props1[0].major_axis_length<200) & (np.abs(props1[0].orientation)<1) & (props1[0].centroid[1]<200 or props1[0].centroid[1]>380 or props1[0].centroid[0]>380):
        quality=0
    else:
        quality=1
    
    return closed, quality

def skeletonization(closed,path,original):
    folder_dir=os.path.dirname(path)
    ds = pydicom.dcmread(path)
    pixspac=ds.PixelSpacing[0]
    
    # Skeletonization
    skeleton=skeletonize(closed)    
    cv2.imwrite(os.path.join(folder_dir,'Original_skeleton.jpg'),skeleton*255)

    # Defining the functions that finds the endpoints where the prunning will start
    def endPoints1(skel):
       
        endpoint1=np.array([[0, 0, 0],
                            [0, 1, 2],
                            [0, 2, 1]])

        endpoint2=np.array([[0, 0, 2],
                            [0, 1, 1],
                            [0, 0, 2]])


        endpoint3=np.array([[2, 0, 0],
                            [1, 1, 0],
                            [2, 0, 0]])

        endpoint4=np.array([[0, 0, 0],
                            [2, 1, 0],
                            [1, 2, 0]])
        
        endpoint5=np.array([[0, 0, 0],      
                            [0, 1, 1],      
                            [1, 0, 1]])

        ep1=mh.morph.hitmiss(skel,endpoint1)
        ep2=mh.morph.hitmiss(skel,endpoint2)
        ep3=mh.morph.hitmiss(skel,endpoint3)
        ep4=mh.morph.hitmiss(skel,endpoint4)
        ep5=mh.morph.hitmiss(skel,endpoint5)
        ep = ep1+ep2+ep3+ep4+ep5

        return ep

    def endPoints2(skel):
        endpoint1=np.array([[1, 0, 0], 
                            [1, 1, 0],
                            [2, 0, 0]])

        endpoint2=np.array([[0, 0, 1],
                            [0, 1, 1],
                            [0, 0, 2]])

        endpoint3=np.array([[0, 1, 0],
                            [1, 1, 0],
                            [2, 0, 0]])

        endpoint4=np.array([[0, 1, 0],
                            [0, 1, 1],
                            [0, 0, 2]])
        
        endpoint5=np.array([[0, 0, 0],
                            [1, 1, 0],
                            [0, 1, 0]])

        ep1=mh.morph.hitmiss(skel,endpoint1)
        ep2=mh.morph.hitmiss(skel,endpoint2)
        ep3=mh.morph.hitmiss(skel,endpoint3)
        ep4=mh.morph.hitmiss(skel,endpoint4)
        ep5=mh.morph.hitmiss(skel,endpoint5)
        ep = ep1+ep2+ep3+ep4+ep5
        return ep
    
    def endPoints3(skel):
        endpoint1=np.array([[1, 0, 0], 
                            [0, 1, 0],
                            [0, 0, 0]])

        endpoint2=np.array([[0, 0, 1],
                            [0, 1, 0],
                            [0, 0, 0]])

        endpoint3=np.array([[0, 0, 1],
                            [0, 1, 1],
                            [0, 0, 0]])

        endpoint4=np.array([[1, 0, 0],
                            [1, 1, 0],
                            [0, 0, 0]])

        endpoint5=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 0, 1]])

        endpoint6=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [1, 0, 0]])

        endpoint7=np.array([[0, 0, 0],
                            [0, 1, 1],
                            [0, 0, 1]])

        endpoint8=np.array([[0, 0, 0],
                            [1, 1, 0],
                            [1, 0, 0]])

        endpoint9=np.array([[0, 0, 0],
                            [0, 1, 1],
                            [0, 1, 0]])

        endpoint10=np.array([[0, 1, 0],
                            [1, 1, 0],
                            [0, 0, 0]])

        endpoint11=np.array([[0, 1, 0],
                            [0, 1, 1],
                            [0, 0, 0]])

        endpoint12=np.array([[0, 0, 0],
                            [1, 1, 0],
                            [0, 1, 0]])

        endpoint13=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 0, 0]])

        endpoint14=np.array([[0, 0, 1],
                            [0, 1, 1],
                            [0, 0, 1]])

        endpoint15=np.array([[1, 0, 0],
                            [1, 1, 0],
                            [1, 0, 0]]) 

        endpoint16=np.array([[0, 1, 1],
                            [0, 1, 0],
                            [0, 0, 0]]) 
    
        endpoint17=np.array([[1, 1, 0],
                            [0, 1, 0],
                            [0, 0, 0]]) 

        endpoint18=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [1, 1, 0]]) 
        
        endpoint19=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 1, 1]]) 
        
        endpoint20=np.array([[0, 0, 0],
                            [0, 1, 0],
                            [1, 1, 0]]) 
    
        ep1=mh.morph.hitmiss(skel,endpoint1)
        ep2=mh.morph.hitmiss(skel,endpoint2)
        ep3=mh.morph.hitmiss(skel,endpoint3)
        ep4=mh.morph.hitmiss(skel,endpoint4)
        ep5=mh.morph.hitmiss(skel,endpoint5)
        ep6=mh.morph.hitmiss(skel,endpoint6)
        ep7=mh.morph.hitmiss(skel,endpoint7)
        ep8=mh.morph.hitmiss(skel,endpoint8)
        ep9=mh.morph.hitmiss(skel,endpoint9)
        ep10=mh.morph.hitmiss(skel,endpoint10)
        ep11=mh.morph.hitmiss(skel,endpoint11)
        ep12=mh.morph.hitmiss(skel,endpoint12)
        ep13=mh.morph.hitmiss(skel,endpoint13)
        ep14=mh.morph.hitmiss(skel,endpoint14)
        ep15=mh.morph.hitmiss(skel,endpoint15)
        ep16=mh.morph.hitmiss(skel,endpoint16)
        ep17=mh.morph.hitmiss(skel,endpoint17)
        ep18=mh.morph.hitmiss(skel,endpoint18)
        ep19=mh.morph.hitmiss(skel,endpoint19)
        ep20=mh.morph.hitmiss(skel,endpoint20)


        ep = ep1+ep2+ep3+ep4+ep5+ep6+ep7+ep8+ep9+ep10+ep11+ep12+ep13+ep14+ep15+ep16+ep17+ep18+ep19+ep20
        return ep
    
    # Function used to prune by applying the first endPoints function    
    def pruning1(skeleton, size):
        '''remove iteratively end points "size" 
           times from the skeleton
        '''
        for i in range(0, size):
            endpoints = endPoints1(skeleton)
            endpoints = np.logical_not(endpoints)
            skeleton = np.logical_and(skeleton,endpoints)
        return skeleton

    # Function used to prune by applying the second endPoints function 
    def pruning2(skeleton, size):
        '''remove iteratively end points "size" 
           times from the skeleton
        '''
        for i in range(0, size):
            endpoints = endPoints2(skeleton)
            endpoints = np.logical_not(endpoints)
            skeleton = np.logical_and(skeleton,endpoints)
        return skeleton
    
    # Function used to prune by applying the third endPoints function 
    def pruning3(skeleton, size):
        '''remove iteratively end points "size" 
           times from the skeleton
        '''
        for i in range(0, size):
            endpoints = endPoints3(skeleton)
            endpoints = np.logical_not(endpoints)
            skeleton = np.logical_and(skeleton,endpoints)
        return skeleton

    # Function that eliminates possible abnormalities in the upper part of the skeleton
    def cleaningtop(branch):
        c=1
        for i in range(len(branch)):
                for j in range(len(branch)):
                    if c<10:
                        if branch[i][j]==1:
                            branch[i][j]=0
                            c+=1
        return branch

    skeleton=cleaningtop(skeleton) # cleaning the top of the skeleton 
    skeleton1 = pruning1(skeleton,40) # First pruning function
    
    # Skeletonization again
    skeleton2=skeletonize(skeleton1) 
    cv2.imwrite(os.path.join(folder_dir,'prunned_skeleton.jpg'),skeleton2*255)
    
    # Showing the overlay of the skeleton on the original image
    
    folder_dir=os.path.dirname(path)
    fig, ax = plt.subplots(figsize=(10,10))
    draw.overlay_skeleton_2d(original, skeleton2, dilate=1,axes=ax)
    plt.show()
    fig.savefig(os.path.join(folder_dir,'superposition_skeleton_original.jpg'))
    plt.close()
   
    # Analyze the skeleton to find the 3 branches
    pixel_graph, coordinates, degrees = skeleton_to_csgraph(skeleton2,spacing=pixspac)
    branch_data_initial = summarize(Skeleton(skeleton2, spacing=pixspac))
    
    # Separate the branches for the labeling
    i=degrees==2
    labeled_branch, nbranches = label(i,return_num=True)
    
    # Removing the small branches in order to remove the attached branches to the main structure
    labeled_branch2=labeled_branch
    index_label=branch_data_initial[branch_data_initial['branch-distance']<26].index.tolist()
        
    if len(index_label)!=0:
        for i in index_label:
            skeleton2[labeled_branch2==i+1]=0
    
    skeleton3=pruning3(skeleton2,3)
    
    cv2.imwrite(os.path.join(folder_dir,'Clean_skeleton.jpg'),skeleton3*255)
    
    pixel_graph2, coordinates2, degrees2 = skeleton_to_csgraph(skeleton3,spacing=pixspac)
    branch_data_final = summarize(Skeleton(skeleton3, spacing=pixspac))
    branch_data_final
    
    branches_of_interest=degrees2==2
    final_labeled, nbranches_final= label(branches_of_interest,return_num=True)

    return final_labeled,degrees2,nbranches

def caracterization(final_labeled,original,path,degrees2):
    folder_dir=os.path.dirname(path)
    ds = pydicom.dcmread(path)
    pixspac=ds.PixelSpacing[0]
    
    # Assign a label to each branch
    aorta=final_labeled==1
    iliac1=final_labeled==2
    iliac2=final_labeled==3
    
    # Identification of the original endpoints and junction points
    endpoints=degrees2==1
    junctionpoint=degrees2==3
    points=endpoints+junctionpoint
    
    folder_dir=os.path.dirname(path)
    fig, ax = plt.subplots(figsize=(10,10))
    draw.overlay_skeleton_2d(original, points, dilate=3,axes=ax)
    plt.show()
    fig.savefig(os.path.join(folder_dir,'Initial_endpoints_and_junction_point.jpg'))
    plt.close()
    
    # Get the coordinates of the pixels belonging to each branch
    def getcoordinates(branch):
        pixel_graph, coordinates, degrees = skeleton_to_csgraph(branch,unique_junctions=False)
        return coordinates
   
    # Function to count how many pixels belong to a branch
    def howmanypixels(branch):
        l=0
        for i in range(len(branch)):
            for j in range(len(branch)):
                if branch[i][j]==1:
                    l+=1
        return l

    # Create a table with the computed Tortuosity measure
    def summary_table (branch):
        pixel_graph, coordinates, degrees = skeleton_to_csgraph(branch,unique_junctions=False)
        branch_data = summarize(Skeleton(branch, spacing=pixspac))    
        branch_data['Tortuosity'] = branch_data.apply(lambda row: row['branch-distance'] - row['euclidean-distance'], axis=1)
        return branch_data
    
    # Location the main bifurcation point
    def assignment2(img,branch):

        position=[]
        for i,e in enumerate(img):
            if 170<i<385:
                for t,r in enumerate(img):
                    if 260<t<380:
                        if img[i][t]==True:
                            position.append([i,t])
                            return position
        if len(position)==3:
            position=position[1]
            return position
        elif len(position)==2:
            position=position[0]
            return position
        elif len(position)==0:
            position.append(getcoordinates(branch)[-1].tolist())
            return position
    
    # Function that classifies the pixels of the segment under study in 2 lists, according to whether they are 
    # diagonally or vertically to their previous neighbours. The exact desired distance of the branch can be also set  

    # For the infrarenal aortic artery
    dstdiagonal=np.sqrt((pixspac**2)+(pixspac**2)) # # diagonal distance in mm between the pixels of the image where pixspac is the space between the centers of neighboring pixels 
    def type_of_neighborsaorta(branch,c):
        branchcoordinates=getcoordinates(branch)
        vertical=[]
        diagonal=[]
        l=c
        index=np.arange(len(branchcoordinates)-1,1,-1) # Turn around the index vector to star the process at the bottom.
        for i in index:
            dx1=(branchcoordinates[i-1][0]-branchcoordinates[i][0])**2
            dx2=(branchcoordinates[i-1][1]-branchcoordinates[i][1])**2
            dst=np.sqrt(dx1+dx2)
            round(dst,4)

            if dst==np.sqrt(2): #if it is square root of 2 it means boths pixels are into a diagonal line
                if l>0:
                    l-=dstdiagonal
                    diagonal.append([branchcoordinates[i-1][0],branchcoordinates[i-1][1]])
                    diagonal.append([branchcoordinates[i][0],branchcoordinates[i][1]])

            elif dst==1: #if it is 1 it means 1 pixel is above the other
                if l>0:
                    l-=pixspac
                    vertical.append([branchcoordinates[i-1][0],branchcoordinates[i-1][1]])
                    vertical.append([branchcoordinates[i][0],branchcoordinates[i][1]])
        return vertical,diagonal,l

    # For the Iliac arteries
    def type_of_neighborsiliac(branch,c):
        branchcoordinates=getcoordinates(branch)
        vertical=[]
        diagonal=[]
        l=c

        for i in range(1,len(branchcoordinates)):
            dx1=(branchcoordinates[i-1][0]-branchcoordinates[i][0])**2
            dx2=(branchcoordinates[i-1][1]-branchcoordinates[i][1])**2
            dst=np.sqrt(dx1+dx2)
            round(dst,4)

            if dst==np.sqrt(2): #if it is square root of 2 it means boths pixels are into a diagonal line
                if l>0:
                    l-=dstdiagonal
                    diagonal.append([branchcoordinates[i-1][0],branchcoordinates[i-1][1]])
                    diagonal.append([branchcoordinates[i][0],branchcoordinates[i][1]])

            elif dst==1: #if it is 1 it means 1 pixel is above the other
                if l>0:
                    l-=pixspac
                    vertical.append([branchcoordinates[i-1][0],branchcoordinates[i-1][1]])
                    vertical.append([branchcoordinates[i][0],branchcoordinates[i][1]])
            elif 2<dst<3:
                if l>0:
                    l-=dstdiagonal
                    diagonal.append([branchcoordinates[i-1][0],branchcoordinates[i-1][1]])
                    diagonal.append([branchcoordinates[i][0],branchcoordinates[i][1]])
        return vertical,diagonal,l
    
   # Function that adjusts the length of the branch by removing the pixels that have not been selected in the previous functions

    # For the Aorta
    def setlength_aorta2(branch):
        indexi=np.arange(branch.shape[0]-1,0,-1) # Start checking the image from the botom of the image to get the first 10 cm
        indexj=np.arange(branch.shape[0]-1,0,-1)
        vertical=type_of_neighborsaorta(branch,100)[0] # Set the desired branch length to 100 mm
        diagonal=type_of_neighborsaorta(branch,100)[1]
        limit=assignment2(junctionpoint,aorta)[0][0]
        for i in indexi:
            for j in indexj:
                    if [i,j] in vertical:
                        if i<limit:
                            branch[i][j]=1
                    elif [i,j] in diagonal:
                        if i<limit:
                            branch[i][j]=1
                    else:
                        branch[i][j]=0
        return branch

    # For the Iliac
    def setlength_iliac2(branch):
        vertical=type_of_neighborsiliac(branch,60)[0] # Set the desired branch length to 60 mm
        diagonal=type_of_neighborsiliac(branch,60)[1]
        for i in range(len(branch)):
            for j in range(len(branch)):
                if [i,j] in vertical:
                        branch[i][j]=1
                elif [i,j] in diagonal:
                        branch[i][j]=1
                else:
                    branch[i][j]=0
        return branch
    
    # Function to plot the analyzed segment with the Tortuosity measure
    def represent1(branch):
        folder_dir=os.path.dirname(path)
        fig, ax = plt.subplots(figsize=(10,10))
        draw.overlay_skeleton_2d(original, branch, dilate=1,axes=ax)
        plt.title('Tortuosity:{}, Length:{} mm'.format(round(summary_table(branch)['Tortuosity'][0],4),round((branch)['branch-distance'][0],2)), fontsize=20)
        plt.show()
        fig.savefig(os.path.join(folder_dir,'Tortuosity_aorta.jpg'))
        plt.close()
    
    def represent2(branch):
        folder_dir=os.path.dirname(path)
        fig, ax = plt.subplots(figsize=(10,10))
        draw.overlay_skeleton_2d(original, branch, dilate=1,axes=ax)
        plt.title('Tortuosity:{}, Length:{} mm'.format(round(summary_table(branch)['Tortuosity'][0],4),round(summary_table(branch)['branch-distance'][0],2)), fontsize=20)
        plt.show()
        fig.savefig(os.path.join(folder_dir,'Tortuosity_R-iliac.jpg'))
        plt.close()
    
    def represent3(branch):
        folder_dir=os.path.dirname(path)
        fig, ax = plt.subplots(figsize=(10,10))
        draw.overlay_skeleton_2d(original, branch, dilate=1,axes=ax)
        plt.title('Tortuosity:{}, Length:{} mm'.format(round(summary_table(branch)['Tortuosity'][0],4),round(summary_table(branch)['branch-distance'][0],2)), fontsize=20)
        plt.show()
        fig.savefig(os.path.join(folder_dir,'Tortuosity_L-iliac.jpg'))
        plt.close()
    
    def represent4(image1,image2,image3):
      final_image=image1+image2+image3 
      folder_dir=os.path.dirname(path)
      fig, ax = plt.subplots(figsize=(10,10))
      draw.overlay_skeleton_2d(original, final_image, color=(1, 0, 0), dilate=1,axes=ax)
      fig.suptitle('Tortuosity Aorta:{}, Length:{} mm \n Tortuosity R-Iliac:{}, Length:{} mm \n Tortuosity L-Iliac:{}, Length:{} mm '.format(round(summary_table(image1)['Tortuosity'][0],4),round(summary_table(image1)['branch-distance'][0],4),round(summary_table(image2)['Tortuosity'][0],4),round(summary_table(image2)['branch-distance'][0],4),round(summary_table(image3)['Tortuosity'][0],4),round(summary_table(image3)['branch-distance'][0],4),2), fontsize=20)
      plt.show()
      fig.savefig(os.path.join(folder_dir,'Tortuosity representation.jpg'))
      plt.close()

    # Aorta
    if howmanypixels(aorta)!=0:
        aortamod=setlength_aorta2(aorta)  
    else:
        aortamod=None
        
    # Iliac    
    if howmanypixels(iliac1) !=0:
        iliac11=setlength_iliac2(iliac1)  
    else:
        iliac11=None
        
    # Iliac
    if howmanypixels(iliac2) !=0:
        iliac22=setlength_iliac2(iliac2)
    else:
        iliac22=None
    
    represent4(aortamod,iliac11,iliac22)
    
    # Overlay of each analyzed branch
    represent1(aortamod)
    plt.close()
    try:
        represent2(iliac11)
        plt.close()
    except:
        pass
        plt.close()
    try:
        represent3(iliac22)
        plt.close()
    except:
        pass  
        plt.close()
    
    # The observations are done according to the branches analyzed
    observacions=''
    if aortamod is not None and iliac11 is not None and iliac22 is not None:
        observacions='Ok'
    elif aortamod is not None and iliac11 is None and iliac22 is None:
        observacions="No s'han analitzat les ilíaques.Revisar possible anomalia o mala reconstrucció"
    elif aortamod is None and iliac11 is None and iliac22 is None:
        observacions="Mala reconstrucció"    
        
    ### EXTRACTION OF THE FINAL POINTS INFORMATION                    
    # Now that the the branches have the desired length, the new endpoints and its coordinates are found

    def getstartingpoint(branch):
        l=0
        p=[]
        for i in range(len(branch)):
            for j in range(len(branch)):
                if l==0:
                    if branch[i][j]==1:
                        l+=1
                        p.append(i)
                        p.append(j)
                else:  
                    branch[i][j]=0    
        return branch,p

    def getfinalpoint(branch):
        indexi=np.arange(branch.shape[0]-1,0,-1) # Start checking the image from the botom 
        indexj=np.arange(branch.shape[0]-1,0,-1)
        l=0
        p=[]
        for i in indexi:
            for j in indexj:
                if l==0:
                    if branch[i][j]==1:
                        l+=1
                        p.append(i)
                        p.append(j)
                else:  
                    branch[i][j]=0    
        return branch,p

    analysis_points=original*0

    # Set the values of the coordinates of each point 

    if aortamod is None:
        p1x='NaN'
        p1y='NaN'
        imagepoint1=None
    else:
        _,_,degrees3 = skeleton_to_csgraph(aortamod,spacing=pixspac)
        endpoints1=degrees3==1
        p1x=getstartingpoint(endpoints1)[1][0]
        p1y=getstartingpoint(endpoints1)[1][1]
        imagepoint1=getstartingpoint(endpoints1)[0]
        analysis_points+=imagepoint1
    if iliac11 is None:
        p2x='NaN'
        p2y='NaN'
        imagepoint2=None
    else: 
        _,_,degrees4 = skeleton_to_csgraph(iliac11,spacing=pixspac)
        endpoints2=degrees4==1
        p2x=getfinalpoint(endpoints2)[1][0]
        p2y=getfinalpoint(endpoints2)[1][1]
        imagepoint2=getfinalpoint(endpoints2)[0]
        analysis_points+=imagepoint2
    if iliac22 is None:
        p3x='NaN'
        p3y='NaN'
        imagepoint3=None
    else:
        _,_,degrees5 = skeleton_to_csgraph(iliac22,spacing=pixspac)
        endpoints3=degrees5==1
        p3x=getfinalpoint(endpoints3)[1][0]
        p3y=getfinalpoint(endpoints3)[1][1]
        imagepoint3=getfinalpoint(endpoints3)[0]
        analysis_points+=imagepoint3
        
    # It will be the junction point in the case that it exists and if not, it will be the endpoint of the aorta
    if aortamod is None and iliac11 is None and iliac22 is None:
        p4x='NaN'
        P3y='NaN'
        imagepoint4=None
    else:
        p4x=assignment2(junctionpoint,aorta)[0][0]
        p4y=assignment2(junctionpoint,aorta)[0][1]
        imagepoint4=original*0
        for i in range(len(imagepoint4)):
            if i==p4x: 
                for j in range(len(imagepoint4)):
                    if j==p4y:
                        imagepoint4[i][j]=True
        analysis_points+=imagepoint4

    # Overlay of the finaly endpoints and junction point

    folder_dir=os.path.dirname(path)
    fig, ax = plt.subplots(figsize=(10,10))
    draw.overlay_skeleton_2d(original, analysis_points, dilate=3,axes=ax)
    plt.show()
    fig.savefig(os.path.join(folder_dir,'OVerlay of the final Endpoints and junction point.jpg'))
    plt.close()
    
    # Set the values for the variables that will fit the final Dataframe with the results
    
    if aortamod is not None:
        s1l=round(summary_table(aortamod)['branch-distance'][0],2)
        s1_t=round(summary_table(aortamod)['Tortuosity'][0],2)
    else:
        s1l='NaN'
        s1_t2='NaN'
    if iliac11 is not None:
        s2l=round(summary_table(iliac11)['branch-distance'][0],2)
        s2_t=round(summary_table(iliac11)['Tortuosity'][0],2)
    else:
        s2l='NaN'
        s2_t='NaN'
    if iliac22 is not None:
        s3l=round(summary_table(iliac22)['branch-distance'][0],2)
        s3_t=round(summary_table(iliac22)['Tortuosity'][0],2)
    else:
        s3l='NaN'
        s3_t='NaN'
        
    # Creation of the Pandas DataFrame
    data={'Patient ID':[ds.PatientID],'p1x':[p1x],'p1y':[p1y],'p2x':[p2x],'p2y':[p2y],'p3x':[p3x],'p3y':[p3y],'p4x':[p4x],'p4y':[p4y],'Aorta-l':[s1l],'Aorta_t2':[s1_t],'R-Iliac-l':[s2l],'R-Iliac_t2':[s2_t],'L-Iliac-l':[s3l],'L_Iliac_t2':[s3_t],'Obs':[observacions]}
    df = pd.DataFrame(data)
    df.index+=1
    
    # Styling the dataframe
    def tortuosity_red(x):
        r = 'background-color: yellow'
        df1 = pd.DataFrame('', index=x.index, columns=x.columns)
        df1.iloc[:, 11] = r
        df1.iloc[:, 14] = r
        df1.iloc[:, 17] = r
        return df1    
    
    df.style.apply(tortuosity_red, axis=None)
    
    # Creation of the CSV. Saving it in the folder with the rest of CSV files to later on generate the final Excel
    identifier=ds.PatientID
    csv_path= "/content/drive/My Drive/CSV"+'/'+identifier+".csv"
    df.to_csv(csv_path,sep=str(','),index=False,header=True,encoding='utf-8') # The header is not exported, only the values
    return df


In [ ]:
# Exuction of the Automatic System in all the files loaded
for case in l:
    aorta, original_image = pre_processing(case)
    closed, cualitat = segmentation(aorta,case)
    if cualitat==0:
        print('Bad reconstruction file: {}'.format(case))
        continue
    else:
        final_labeled, degrees2,nbranches = skeletonization(closed,case,original_image)
        if nbranches>15:
            print('Bad reconstruction file: {}'.format(case))
            continue
        else:
            results = caracterization(final_labeled,original_image,case,degrees2)
            print(results)